In [82]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
tf.__version__

'2.7.0'

In [83]:
batch_size = 10
split = 0.3
epochs = 55

In [84]:
class fotw_dataframe():
    '''Wraps the celebA dataset, allowing an easy way to:
       - Select the features of interest,
       - Split the dataset into 'training', 'test' or 'validation' partition.
    '''
    def __init__(self,attr_file = "gender_fex_trset.csv", images_file = 'gender_smile', main_folder = "..\data\\fotw" , selected_features = [] , drop_features = [],):
        self.main_folder = main_folder
        self.images_folder   = os.path.join(main_folder, images_file)
        self.attr_path = os.path.join(self.images_folder, attr_file)
        self.features_name = []
        self.prepare(drop_features , selected_features)
        
    def prepare(self,drop_features, selected_features ):
        
        #attributes selection
        if len(selected_features) == 0  :
            self.attributes = pd.read_csv(self.attr_path)
            self.num_features = 2
        else:
            self.num_features = len(selected_features)
            selected_features = selected_features.copy()
            selected_features.append("image_name")
            self.attributes = pd.read_csv(self.attr_path,usecols=selected_features)
        
        #removing features
        if len(drop_features) != 0:
            for feature in drop_features:
                if feature in self.attributes:
                    self.attributes = self.attributes.drop(feature , axis = 1)
                    self.num_features -= 1
            
        self.attributes.set_index("image_name" , inplace = True)
        self.attributes.replace(to_replace = -1 , value = 0 , inplace = True)
        self.attributes["image_name"] = list(self.attributes.index)
        self.features_name = list(self.attributes.columns)[:-1]
        
        return self.attributes 

In [85]:
#features  = ['Gender','Smile']
features = ['Gender']
fotw_df = fotw_dataframe()
fotw = fotw_df.prepare(drop_features=[],selected_features= features) 
fotw

,Gender,image_name
image_name,,
im_GenFex_00001.jpg,0,im_GenFex_00001.jpg
im_GenFex_00002.jpg,1,im_GenFex_00002.jpg
im_GenFex_00003.jpg,0,im_GenFex_00003.jpg
im_GenFex_00004.jpg,1,im_GenFex_00004.jpg
im_GenFex_00005.jpg,1,im_GenFex_00005.jpg
...,...,...
im_GenFex_09253.jpg,1,im_GenFex_09253.jpg
im_GenFex_09254.jpg,1,im_GenFex_09254.jpg
im_GenFex_09255.jpg,0,im_GenFex_09255.jpg


In [86]:
split_ind = int((1 - split) * fotw.shape[0])
train_df = fotw[:split_ind]
validation_df = fotw[split_ind:]
train_df,validation_df

(                     Gender           image_name
 image_name                                      
 im_GenFex_00001.jpg       0  im_GenFex_00001.jpg
 im_GenFex_00002.jpg       1  im_GenFex_00002.jpg
 im_GenFex_00003.jpg       0  im_GenFex_00003.jpg
 im_GenFex_00004.jpg       1  im_GenFex_00004.jpg
 im_GenFex_00005.jpg       1  im_GenFex_00005.jpg
 ...                     ...                  ...
 im_GenFex_06475.jpg       0  im_GenFex_06475.jpg
 im_GenFex_06476.jpg       0  im_GenFex_06476.jpg
 im_GenFex_06477.jpg       0  im_GenFex_06477.jpg
 im_GenFex_06478.jpg       1  im_GenFex_06478.jpg
 im_GenFex_06479.jpg       0  im_GenFex_06479.jpg
 
 [6479 rows x 2 columns],
                      Gender           image_name
 image_name                                      
 im_GenFex_06480.jpg       0  im_GenFex_06480.jpg
 im_GenFex_06481.jpg       0  im_GenFex_06481.jpg
 im_GenFex_06482.jpg       0  im_GenFex_06482.jpg
 im_GenFex_06483.jpg       1  im_GenFex_06483.jpg
 im_GenFex_06484.jpg  

In [87]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# augumentations for training set:
train_datagen = ImageDataGenerator(rescale=1./255)

valid_datagen = ImageDataGenerator(rescale= 1./255)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory=fotw_df.images_folder,
    x_col='image_name',
    y_col=fotw_df.features_name,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='raw',
    shuffle = True
)

validation_generator = valid_datagen.flow_from_dataframe(dataframe=validation_df,
                                                         directory=fotw_df.images_folder,
                                                         x_col='image_name',
                                                         y_col=fotw_df.features_name,
                                                         target_size=(224, 224),
                                                         batch_size=batch_size,
                                                         class_mode='raw'
                                                        )


Found 6479 validated image filenames.
Found 2778 validated image filenames.


In [88]:
from tensorflow.keras.applications.mobilenet_v2  import MobileNetV2
from tensorflow.keras.layers import Dense , BatchNormalization , Dropout

def classifier(num_features):
    base = MobileNetV2(input_shape = (224,224,3),
                      weights = 'imagenet',
                      include_top=False,
                      pooling = "avg")
    
    x = base.output
    x = Dense(1536, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.29)(x)
    top = Dense(num_features, activation='sigmoid')(x)
    classifier = tf.keras.models.Model(base.input,top)
    
    return classifier

In [89]:
num_features = len(fotw_df.features_name)
cls = classifier(num_features)
cls.summary()

Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_9 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_9[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                           

In [90]:
cls.compile(loss='binary_crossentropy',
              optimizer= tf.keras.optimizers.SGD(learning_rate=0.0001),
              metrics='binary_accuracy')

In [91]:
from tensorflow.keras.callbacks import EarlyStopping , ModelCheckpoint

earlystop = EarlyStopping(monitor="val_binary_accuracy", patience= 3)
checkpoint_filepath = "..\data" + f"/weights-fotw{fotw_df.num_features}-MobileNetV2" + "{val_binary_accuracy:.2f}.hdf5"
model_checkpoint = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_binary_accuracy',
    mode='max',
    save_best_only=True)

In [92]:
len(validation_generator)
history = cls.fit(
    train_generator,
    epochs=epochs,
    steps_per_epoch=len(train_generator),
    validation_data=validation_generator,
    validation_steps=len(validation_generator),
    callbacks = [model_checkpoint],
    max_queue_size=1,
    verbose=1)

Epoch 1/55
648/648 [==============================] - 87s 127ms/step - loss: 0.7832 - binary_accuracy: 0.5819 - val_loss: 0.6390 - val_binary_accuracy: 0.6760
Epoch 2/55
648/648 [==============================] - 88s 135ms/step - loss: 0.6420 - binary_accuracy: 0.6692 - val_loss: 0.5852 - val_binary_accuracy: 0.7059
Epoch 3/55
648/648 [==============================] - 84s 129ms/step - loss: 0.5948 - binary_accuracy: 0.6925 - val_loss: 0.5632 - val_binary_accuracy: 0.7153
Epoch 4/55
648/648 [==============================] - 85s 131ms/step - loss: 0.5574 - binary_accuracy: 0.7180 - val_loss: 0.5454 - val_binary_accuracy: 0.7264
Epoch 5/55
648/648 [==============================] - 84s 129ms/step - loss: 0.5431 - binary_accuracy: 0.7285 - val_loss: 0.5322 - val_binary_accuracy: 0.7376
Epoch 6/55
648/648 [==============================] - 82s 126ms/step - loss: 0.5143 - binary_accuracy: 0.7413 - val_loss: 0.5289 - val_binary_accuracy: 0.7340
Epoch 7/55
648/648 [==========================

In [93]:
#path = "model\\fotw"
#cls.save_weights('model\\fotw_weights')

In [94]:
#n_cls = classifier(40)
#n_cls.load_weights("..\data\weights-FC40-MobileNetV20.87.hdf5")